# Download NMR Dart survey data

**Author:** Logan Schmidt

**Created:** 2019-06-07

**Last Updated:** 2019-06-09

**Purpose:**

This script was created as part of the effort to produce the **Schmidt, Rempe 2019 Water Content** manuscript. This script accesses the NMR Dart survey data saved on the **DRempe Drive** on the JSG server and imports, processes, and saves the data as a single master csv file.

In [71]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import os
import re
from time import gmtime, strftime

In [72]:
# select processed NMR folder to get directory
root = tk.Tk()
importDir = filedialog.askdirectory(title = 'Select Processed NMR Folder')
root.mainloop()
if len(importDir) > 0:
    print('You chose: %s' % importDir)

You chose: /Volumes/DRempe Drive/Logan Schmidt/NMR/DATA/ERCZO/Processed_All/Processed_Survey


In [75]:
# get sitename for processed NMR folder
site = re.findall('/DATA/(\w*)', importDir)[0]
# get pathname for processed NMR folder
rootDir = os.path.abspath(importDir)
# create empty dataframe on which to append individual dataframes to create master dataframe
df2 = pd.DataFrame()

# walk through folders in NMR processed folder and import 1Dvector data files into a master dataframe
for (dirName, dirs, files) in os.walk(rootDir):
   for fileName in files:
       if fileName.startswith('well') & fileName.endswith('_1Dvectors_uniform.txt'):
            # get the name of the 1Dvectors file
            fileCrit = os.path.join(dirName,fileName)            
            # convert the *_1Dvectors_uniform.txt file into a dataframe
            df1 = pd.read_csv(fileCrit, sep = '\s+')

            # disect fileName to get wellID and year-month-day
            exps0  = re.findall('/(\w*)', fileCrit)
            exps1  = re.findall('(\w*?)_', exps0[-1])
            wellID = exps1[1]
            year   = exps1[2]
            month  = exps1[3]
            day    = exps1[4]
            if day == '1Dvectors':
                day = '01'
            datetime = pd.to_datetime(year + month + day)

            # add wellID and date vectors
            df1.insert(0, 'wellID', wellID, True)
            df1.insert(1, 'date', datetime, True)
            
            # create multi-indecies
#             df1.set_index(['wellID', 'date', 'depth'], inplace = True)
            
            df2 = df2.append(df1)

In [76]:
# get current datetime to append to filename
datetime = strftime("%Y-%m-%d-%H%M-UTC", gmtime())

# Save *working file*
df2.to_csv('Data/NMRDartSurveyRaw_' + site + '.csv', 
           index = False)
# Save time-stamped *back-up file*
df2.to_csv('Data/NMRDartSurveyRaw_' + site + '/Old/' + datetime + ' NeutronandGammaMasterRaw.csv', 
           index = False)

# Read it in and check
pd.read_csv('Data/NMRDartSurveyRaw_' + site + '.csv',
            sep = ',',  
            skiprows = [1]).head()

,wellID,date,depth,totalf,clayf,capf,freef,mlT2,Ksdr,Ktc,Ksoe,Tsdr,Ttc,Tsoe,soe,noise
0,508,2017-10-13,1.50,0.143974,0.104496,0.013730,0.025749,0.003187,0.013014,1.700000e-08,0.082071,0.003254,4.300000e-09,0.020518,0.000050,3.637994
1,502,2017-08-01,3.00,0.132918,0.098594,0.009194,0.025130,0.003574,0.015112,1.420000e-08,0.228294,0.048059,8.000000e-09,0.273320,0.000114,3.254857
2,502,2017-08-01,2.75,0.125618,0.091807,0.012819,0.020992,0.003666,0.015029,8.400000e-09,0.197633,0.044281,4.500000e-09,0.216247,0.000073,3.086515
3,502,2017-08-01,2.50,0.114175,0.026551,0.071272,0.016351,0.009024,0.082751,4.000000e-09,0.339734,0.040523,2.400000e-09,0.166838,0.000146,3.284373
4,502,2017-08-01,2.25,0.133396,0.069202,0.052091,0.012102,0.004663,0.025810,2.600000e-09,0.219815,0.019836,1.400000e-09,0.081905,0.000094,3.242540


array([('508', Timestamp('2017-10-13 00:00:00'), 1.75),
       ('508', Timestamp('2017-10-13 00:00:00'), 1.5),
       ('502', Timestamp('2017-08-01 00:00:00'), 3.0), ...,
       ('005', Timestamp('2017-08-13 00:00:00'), 2.0),
       ('005', Timestamp('2017-08-13 00:00:00'), 1.75),
       ('005', Timestamp('2017-08-13 00:00:00'), 1.5)], dtype=object)